# Natural Computing Project - BiomakerCA - Simulating Plant Reproducibility 

### 1. Overrides

In [1]:
# Overriding the default environment logic with a custom one
import overrides.env_logic_override as env_override
import self_organising_systems.biomakerca.env_logic as env_logic
env_logic.process_energy = env_override.process_energy


# Overriding the default step maker with a custom one
import overrides.step_maker_override as step_maker_override
import self_organising_systems.biomakerca.step_maker as step_maker
step_maker.step_env = step_maker_override.step_env



### 2. Imports

In [2]:
import jax.random as jr
import mediapy as media
from IPython.display import Video
from jax import vmap
from self_organising_systems.biomakerca import environments as evm
from self_organising_systems.biomakerca.agent_logic import BasicAgentLogic
from self_organising_systems.biomakerca.mutators import (
    BasicMutator, RandomlyAdaptiveMutator)

from biomaker_utils import perform_evaluation, perform_simulation, start_simulation
from configs.seasons_config import SeasonsConfig as Config



### 3. Baseline Configuration

In [3]:
config = Config()

st_env, env_config = evm.get_env_and_config(config.ec_id, width_type=config.env_width_type)
env_config.soil_unbalance_limit = config.soil_unbalance_limit
agent_logic = BasicAgentLogic(env_config, minimal_net=config.agent_model == "minimal")

sd = 1e-2 if config.mutator_type == "basic" and config.agent_model == "basic" else 1e-3
mutator = (
    BasicMutator(sd=sd, change_perc=0.2)
    if config.mutator_type == "basic"
    else RandomlyAdaptiveMutator(init_sd=sd, change_perc=0.2)
)

print("\n\nCurrent config:")
print("\n".join("%s: %s" % item for item in vars(env_config).items()))

ku, key = jr.split(config.key)
programs = vmap(agent_logic.initialize)(jr.split(ku, config.n_max_programs))
programs = vmap(mutator.initialize)(jr.split(ku, programs.shape[0]), programs)

env = st_env

BasicAgentLogic.dsm_num_params = 0
BasicAgentLogic.nsl_num_params = 176
BasicAgentLogic.denm_num_params = 80
BasicAgentLogic.excl_num_params = 41
BasicAgentLogic.repr_num_params = 2
BasicAgentLogic.num_params = 299


Current config:
agent_state_size: 2
etd: DefaultTypeDef: {materials_list: ['VOID', 'AIR', 'EARTH', 'IMMOVABLE', 'SUN', 'OUT_OF_BOUNDS'], types: {'VOID': 0, 'AIR': 1, 'EARTH': 2, 'IMMOVABLE': 3, 'SUN': 4, 'OUT_OF_BOUNDS': 5, 'AGENT_UNSPECIALIZED': 6, 'AGENT_ROOT': 7, 'AGENT_LEAF': 8, 'AGENT_FLOWER': 9}, specialization_idxs: {'AGENT_UNSPECIALIZED': 0, 'AGENT_ROOT': 1, 'AGENT_LEAF': 2, 'AGENT_FLOWER': 3}, agent_types: [6 7 8 9], intangible_mats: [0 1], gravity_mats: [2 6 7 8 9], structural_mats: [6 7 8 9], propagate_structure_mats: [2 3 6 7 8 9], agent_spawnable_mats: [0 1 2], structure_decay_mats: [-1 -1  1  0 -1 -1  5  5  5  5], aging_mats: [6 7 8 9], dissipation_rate_per_spec: [[0.5 0.5],  [1.  1. ],  [1.  1. ],  [1.2 1.2]]}
env_state_size: 6
struct_integrity_cap: 200
abso

In [4]:
### 4. Performing Basic Simulation

In [5]:
# spring_agent_logic = agent_logic

# frame = start_simulation(env, config, env_config)
# with media.VideoWriter(
# 	config.out_file, shape=frame.shape[:2], fps=config.fps, crf=18
# ) as video:
# 	perform_simulation(
# 		env, programs, config, env_config, spring_agent_logic, mutator, key, video, frame
# 	)

# Video(config.out_file)

In [6]:
# perform_evaluation(env, programs, st_env, env_config, agent_logic, mutator, config)

### 5. Simulating Seasons

In [7]:
APPEND_ALL_SEASONS_IN_ONE_VIDEO = False

#### 5.1 Spring

In [8]:
config.AIR_DIFFUSION_RATE = 0.13
config.SOIL_DIFFUSION_RATE = 0.13

if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
	config.out_file = "spring.mp4"

frame = start_simulation(env, config, env_config)
with media.VideoWriter(
	config.out_file, shape=frame.shape[:2], fps=config.fps, crf=18
) as video:
	step = perform_simulation(
		env, programs, config, env_config, agent_logic, mutator, key, video, frame, season="Spring"
	)

# if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
Video(config.out_file)

Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>
Traced<ShapedArray(float32[])>with<DynamicJaxprTrace(level=1/0)>


  0%|          | 0/500 [00:00<?, ?it/s]/home/janneke/anaconda3/envs/NaCo3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:96: FutureWarning: scatter inputs have incompatible types: cannot safely cast value from dtype=int32 to dtype=uint32 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.
  warnings.warn(
2024-04-30 11:48:48.439150: E external/xla/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  %reduce.78 = f32[72,128,9,6]{3,2,1,0} reduce(f32[1,72,128,9,6]{4,3,2,1,0} %broadcast.30, f32[] %constant.186), dimensions={0}, to_apply=%region_135.8003, metadata={op_name="jit(step_env)/jit(main)/reduce_sum[axes=(0,)]" source_file="/home/janneke/Documents/Master/Natural Computing/NaCO_project/overrides/step_maker_override.py" source_line=157}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime. XLA has some guards that attempt to kee

#### 5.2 Summer

In [9]:
# config.AIR_DIFFUSION_RATE = 0.1
# config.SOIL_DIFFUSION_RATE = 0.1

# if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
# 	config.out_file = "summer.mp4"
# 	frame = start_simulation(env, config, env_config)
# with media.VideoWriter(
# 	config.out_file, shape=frame.shape[:2], fps=config.fps, crf=18
# ) as video:
# 	step = perform_simulation(
# 		env, programs, config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Summer"
# 	)

# # if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
# Video(config.out_file)

#### 5.3 Autumn

In [10]:
# config.AIR_DIFFUSION_RATE = 0.13

# if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
# 	config.out_file = "autumn.mp4"
# 	frame = start_simulation(env, config, env_config)
# with media.VideoWriter(
# 	config.out_file, shape=frame.shape[:2], fps=config.fps, crf=18
# ) as video:
# 	step = perform_simulation(
# 		env, programs, config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Autumn"
# 	)

# # if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
# Video(config.out_file)

#### 5.4 Winter

In [14]:
config.AIR_DIFFUSION_RATE = 0.1
config.SOIL_DIFFUSION_RATE = 0.01

if not APPEND_ALL_SEASONS_IN_ONE_VIDEO:
	config.out_file = "winter.mp4"
	frame = start_simulation(env, config, env_config)
with media.VideoWriter(
	config.out_file, shape=frame.shape[:2], fps=config.fps, crf=18
) as video:
	step = perform_simulation(
		env, programs, config, env_config, agent_logic, mutator, key, video, frame, step = step if APPEND_ALL_SEASONS_IN_ONE_VIDEO else 0, season="Winter"
	)

Video(config.out_file)

  0%|          | 0/500 [02:16<?, ?it/s]


KeyboardInterrupt: 